In [1]:
import pandas as pd
import os, sys
from pathlib import Path

In [22]:
root = Path("../../data/exp")
equi_brightness_meta = [
    # dist (cm), volts, amps
    (20,    45.2,  0.6900),
    (10,    44.7,  0.2400),
    (5,     44.2,  0.1100),
    (3,     44.2,  0.0800),
    (1,     44,    0.0500),
    (0.1,   43.9,  0.0400),
]
equi_brightness_meta = {float(d):(float(v), float(a)) for d, v, a in equi_brightness_meta}

rows = []
orig_cols: list|None = None
for f in os.listdir(root):
    fpath = root.joinpath(f)

    if "e" in f:
        _dist = float(f[1:-4])
        _volts, _amps = equi_brightness_meta.get(_dist, (None, None))
    else:
        _dist = float(f[:-4])
        _volts, _amps = 44.3, 0.09

    # dist = f.replace("")
    _df = pd.read_csv(fpath)
    def parse_coord(col: str):
        x, y = [int(v) for v in col.split("_")]
        return x, y
    coords = [parse_coord(col) for col in _df.columns[2:]]
    if orig_cols is None: orig_cols = [c for c in _df.columns]
    for _, row in _df.iterrows():
        _dt, _frame = row[:2]
        for i, v in enumerate(row[2:]):
            x, y = coords[i]
            rows.append([_dist, _volts, _amps, _dt, _frame, x, y, v])
assert orig_cols is not None    

def _set_type(df: pd.DataFrame, col: str, t):
    df[col] = df[col].astype(t)

cols = "distance_cm, volts, amps".split(", ")
cols += "delta_time_miliseconds, frame, sensor_x, sensor_y, sensor_value".split(", ")
df = pd.DataFrame(rows, columns=cols)
_set_type(df, "delta_time_miliseconds", int)
_set_type(df, "frame", int)
_set_type(df, "sensor_x", int)
_set_type(df, "sensor_y", int)

In [23]:
print(df.shape)
df.to_csv("../../data/exp_cleaned/sensor_outputs.csv")
df.head(16)

(69216, 8)


,distance_cm,volts,amps,delta_time_miliseconds,frame,sensor_x,sensor_y,sensor_value
0,10.0,44.3,0.09,0,1,1,1,58.85
1,10.0,44.3,0.09,0,1,1,2,67.38
2,10.0,44.3,0.09,0,1,1,3,72.31
3,10.0,44.3,0.09,0,1,1,4,72.62
4,10.0,44.3,0.09,0,1,1,5,71.15
5,10.0,44.3,0.09,0,1,1,6,61.00
6,10.0,44.3,0.09,0,1,2,1,63.92
7,10.0,44.3,0.09,0,1,2,2,72.08
8,10.0,44.3,0.09,0,1,2,3,80.08
9,10.0,44.3,0.09,0,1,2,4,82.31


In [24]:
cdf = pd.read_csv("../../data/exp_cleaned/spectrophotometer.csv")
print(cdf.shape)
cdf.head()

(23, 5)


,distance_cm,volts,amps,ppfd_400-700nm,pfd_380-780nm
0,NaN,0.0,0.00,0.0068,0.0105
1,NaN,0.0,0.00,0.0584,0.0927
2,NaN,0.0,0.00,0.0730,0.1161
3,20.0,44.3,0.09,62.1200,66.2700
4,20.0,44.3,0.09,61.9500,66.0800
